# Импорт библиотек

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from official.nlp import optimization

# Чтение датасета

In [ ]:
data = pd.read_csv('dataset.csv')

texts = data['annotation']
labels = data["labels"]

## Разделение на обучающую и тестовую выборки

In [ ]:
train_text, test_text, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Инициализация собственного Callback класса и функции создания модели

In [ ]:
class SaveBestModel(tf.keras.callbacks.Callback):
    """Callback класс для сохранения модели"""
    best_weights = None

    def __init__(self, save_best_metric='val_loss', search_max=False):
        """
        Магический метод инициализации класса

        :param save_best_metric: название метрики для определения лучших весов
        :param search_max: направление сравнения (если True, то сохраняет при наибольшем значении метрики)
        """
        self.metric_arr = []
        self.save_best_metric = save_best_metric
        self.max = search_max
        if search_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        """
        Переопределенная функция tf.keras.callbacks.Callback, вызывается в конце эпохи

        :param epoch: номер эпохи
        :param logs: метрики
        """
        # Получение метрики
        metric_value = logs[self.save_best_metric]
        # Сохранение метрики
        self.metric_arr.append(logs)
        # Проверка условий для сохранения модели
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_weights = self.model.get_weights()
        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_weights = self.model.get_weights()


def build_classifier_model(num_class: int):
    """
    Функция создания архитектуры модели

    :param num_class: количество выходных классов
    """
    # Слой входа
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    # Слой препроцессинга для BERT
    preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3",
                                         name='preprocessing')
    # Препроцессинг
    encoder_inputs = preprocessing_layer(text_input)
    # Слой модели BERT
    encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4", trainable=True,
                             name='BERT')
    # Выход BERT
    outputs = encoder(encoder_inputs)
    # Собственные слои для классификации
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(num_class, activation="softmax", name='classifier')(net)
    return tf.keras.Model(text_input, net)

# Инициализация модели

In [ ]:
model = build_classifier_model(num_class=69)
model.summary()

# Инициализация опитимизатора AdamW

In [ ]:
# Количество эпох
epochs = 40
# Сокрость обучения
init_lr = 3e-5

# Количество шагов в эпоху
steps_per_epoch = tf.data.experimental.cardinality(tf.data.Dataset.from_tensor_slices({"input": train_text, "output": train_labels})).numpy()
# Количество шагов при обучении
num_train_steps = steps_per_epoch * epochs
# Количество warmup шагов
num_warmup_steps = int(0.1 * num_train_steps)

# Инициализация опитимизатора
optimizer = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type='adamw'
)

# Компиляция модели

In [ ]:
# Инициализация callback-а для сохранения лучших весов
save_best_model = SaveBestModel()

# Компиляция модели
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    metrics=['accuracy']
)

# Обучение модели

In [ ]:
model.fit(
    train_text,
    train_labels,
    callbacks=[save_best_model],
    validation_data=(test_text, test_labels),
    epochs=epochs,
    batch_size=25
)

# Тестирование модели

In [ ]:
# Загрузка и сохранение лучших весов
model.set_weights(save_best_model.best_weights)
model.save_weights('GRNTIClassifier_BERT_v1.h5')

In [ ]:
# Пердсказание классов
predict = model.predict(test_text)
pred_arr = [tf.argmax(el).numpy() for el in predict]

In [ ]:
# Настройка вывода numpy
np.set_printoptions(edgeitems=np.inf)
# Вывод матрицы потерь
print(confusion_matrix(test_labels, pred_arr))

In [ ]:
# Вывод репорта классификации
print(classification_report(test_labels, pred_arr))